# Kaggle 
# Sentiment Analysis

### I will first look into a basic prediction model for review type (good or bad)
### Then I will look into developing a ***Subject Finder***
    - This will hopefully give a classification subject for each review (movie,restaurant,item)

In [1]:
import string
import numpy as np
import pandas as pd
from gensim import corpora
from nltk.corpus import stopwords,genesis
from nltk import wordpunct_tokenize,FreqDist
from sklearn.model_selection import train_test_split
from nltk.collocations import TrigramAssocMeasures,TrigramCollocationFinder

C:\Users\Scott\Anaconda3\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [2]:
amazon = pd.read_fwf('amazon_cells_labelled.txt',header=None)
amazon.columns = ['Review']
print(len(amazon))
amazon ['subject'] = 'item'
amazon.head()

1000


,Review,subject
0,So there is no way for me to plug it in here i...,item
1,"Good case, Excellent value.\t1",item
2,Great for the jawbone.\t1,item
3,Tied to charger for conversations lasting more...,item
4,The mic is great.\t1,item


In [3]:
imdb = pd.read_fwf('imdb_labelled.txt',header=None)
imdb.columns = ['Review','0','1','2','3']
imdb = imdb.drop(['0','1','2','3'],axis=1)
imdb ['subject'] = 'movie'
print(len(imdb))
imdb.head()

1000


,Review,subject
0,"A very, very, very slow-moving, aimless movie ...",movie
1,Not sure who was more lost - the flat characte...,movie
2,Attempting artiness with black & white and cle...,movie
3,Very little music or anything to speak of. \t0,movie
4,The best scene in the movie was when Gerardo i...,movie


In [4]:
yelp = pd.read_fwf('yelp_labelled.txt',header=None)
yelp.columns = ['Review','1','2']
yelp = yelp.drop(['1','2'],axis=1)
yelp ['subject'] = 'restaurant'
print(len(yelp))
yelp.head()

1000


,Review,subject
0,Wow... Loved this place.\t1,restaurant
1,Crust is not good.\t0,restaurant
2,Not tasty and the texture was just nasty.\t0,restaurant
3,Stopped by during the late May bank holiday of...,restaurant
4,The selection on the menu was great and so wer...,restaurant


In [5]:
frames = [amazon, yelp, imdb]
df = pd.concat(frames,ignore_index=True)
df = df[:-1]
len(df)

2999

In [6]:
df.Review[-1:]

2998    Exceptionally bad!  \t0
Name: Review, dtype: object

In [7]:
def create_word_features(review):
    lower_review = review.lower()
    words = wordpunct_tokenize(lower_review)
    useful_words = [w for w in words 
                    if w not in string.punctuation 
                    and w not in stopwords.words('english')]
    useful_words = useful_words[:-1]
    return useful_words

def create_all_words(review):
    lower_review = review.lower()
    words = wordpunct_tokenize(lower_review)
    all_words = [w for w in words 
                    if w not in string.punctuation]
    all_words = all_words[:-1]
    return all_words

def move_sentiment(number):
    words = wordpunct_tokenize(number)
    return words[-1]
df ['sentiment'] = df.Review.apply(move_sentiment)
df ['obj_words'] = df.Review.apply(create_word_features)
df ['all_words'] = df.Review.apply(create_all_words)

In [8]:
df = df.loc[df.sentiment.isin(['1','0'])]
df.sentiment = df.sentiment.astype(int)

##### Need to find correlation between group of words and sentiment

In [9]:
df ['obj_word_count'] = df.obj_words.apply(lambda x: len(x))
df ['total_word_count'] = df.all_words.apply(lambda x: len(x))
df.head()

,Review,subject,sentiment,obj_words,all_words,obj_word_count,total_word_count
0,So there is no way for me to plug it in here i...,item,0,"[way, plug, us, unless, go, converter]","[so, there, is, no, way, for, me, to, plug, it...",6,21
1,"Good case, Excellent value.\t1",item,1,"[good, case, excellent, value]","[good, case, excellent, value]",4,4
2,Great for the jawbone.\t1,item,1,"[great, jawbone]","[great, for, the, jawbone]",2,4
3,Tied to charger for conversations lasting more...,item,0,"[tied, charger, conversations, lasting, 45, mi...","[tied, to, charger, for, conversations, lastin...",9,13
4,The mic is great.\t1,item,1,"[mic, great]","[the, mic, is, great]",2,4


In [10]:
import nltk, string, numpy

stemmer = nltk.stem.porter.PorterStemmer()
def StemTokens(tokens):
    return [stemmer.stem(token) for token in tokens]
lemmer = nltk.stem.WordNetLemmatizer()
def LemTokens(tokens):
    return [lemmer.lemmatize(token) for token in tokens]

df['lem_words'] = df.obj_words.apply(LemTokens)
df['stem_words'] = df.obj_words.apply(StemTokens)

In [11]:
df.head()

,Review,subject,sentiment,obj_words,all_words,obj_word_count,total_word_count,lem_words,stem_words
0,So there is no way for me to plug it in here i...,item,0,"[way, plug, us, unless, go, converter]","[so, there, is, no, way, for, me, to, plug, it...",6,21,"[way, plug, u, unless, go, converter]","[way, plug, us, unless, go, convert]"
1,"Good case, Excellent value.\t1",item,1,"[good, case, excellent, value]","[good, case, excellent, value]",4,4,"[good, case, excellent, value]","[good, case, excel, valu]"
2,Great for the jawbone.\t1,item,1,"[great, jawbone]","[great, for, the, jawbone]",2,4,"[great, jawbone]","[great, jawbon]"
3,Tied to charger for conversations lasting more...,item,0,"[tied, charger, conversations, lasting, 45, mi...","[tied, to, charger, for, conversations, lastin...",9,13,"[tied, charger, conversation, lasting, 45, min...","[tie, charger, convers, last, 45, minut, major..."
4,The mic is great.\t1,item,1,"[mic, great]","[the, mic, is, great]",2,4,"[mic, great]","[mic, great]"


In [12]:
good_reviews = df.loc[df.sentiment == 1]
bad_reviews = df.loc[df.sentiment == 0]

In [13]:
good_word_bank = []
for row in good_reviews.stem_words:
    for word in row:
        good_word_bank.append(word)

bad_word_bank = []
for row in bad_reviews.stem_words:
    for word in row:
        bad_word_bank.append(word)
good_freq = FreqDist(good_word_bank)
bad_freq = FreqDist(bad_word_bank)
good_freq.most_common(10)

[('great', 197),
 ('good', 172),
 ('film', 114),
 ('phone', 93),
 ('movi', 93),
 ('work', 83),
 ('love', 82),
 ('one', 71),
 ('like', 68),
 ('well', 61)]

In [14]:
df.obj_words.head()

0               [way, plug, us, unless, go, converter]
1                       [good, case, excellent, value]
2                                     [great, jawbone]
3    [tied, charger, conversations, lasting, 45, mi...
4                                         [mic, great]
Name: obj_words, dtype: object

In [15]:
def create_word_features (words):
    my_dict = dict([(word, True) for word in words])
    return my_dict

df['word_list'] = df.obj_words.apply(create_word_features)

df.head()

,Review,subject,sentiment,obj_words,all_words,obj_word_count,total_word_count,lem_words,stem_words,word_list
0,So there is no way for me to plug it in here i...,item,0,"[way, plug, us, unless, go, converter]","[so, there, is, no, way, for, me, to, plug, it...",6,21,"[way, plug, u, unless, go, converter]","[way, plug, us, unless, go, convert]","{'way': True, 'plug': True, 'us': True, 'unles..."
1,"Good case, Excellent value.\t1",item,1,"[good, case, excellent, value]","[good, case, excellent, value]",4,4,"[good, case, excellent, value]","[good, case, excel, valu]","{'good': True, 'case': True, 'excellent': True..."
2,Great for the jawbone.\t1,item,1,"[great, jawbone]","[great, for, the, jawbone]",2,4,"[great, jawbone]","[great, jawbon]","{'great': True, 'jawbone': True}"
3,Tied to charger for conversations lasting more...,item,0,"[tied, charger, conversations, lasting, 45, mi...","[tied, to, charger, for, conversations, lastin...",9,13,"[tied, charger, conversation, lasting, 45, min...","[tie, charger, convers, last, 45, minut, major...","{'tied': True, 'charger': True, 'conversations..."
4,The mic is great.\t1,item,1,"[mic, great]","[the, mic, is, great]",2,4,"[mic, great]","[mic, great]","{'mic': True, 'great': True}"


In [16]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df.Review,df.sentiment,test_size = .33)

In [17]:
from sklearn.feature_extraction.text import CountVectorizer,TfidfTransformer
count_vect = CountVectorizer()
x_train_counts = count_vect.fit_transform(X_train)
tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(x_train_counts)

x_test_counts = count_vect.transform(X_test)
x_test_tfidf = tfidf_transformer.transform(x_test_counts)

In [18]:
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB().fit(X_train_tfidf,y_train)
clf.score(x_test_tfidf,y_test)

0.8223140495867769

### Pipelines

In [19]:
from sklearn.pipeline import Pipeline
from sklearn.model_selection import cross_val_score,ShuffleSplit

text_clf = Pipeline([('vect', CountVectorizer()),
                    ('tfidf',TfidfTransformer()),
                    ('clf', MultinomialNB()),])
cv = ShuffleSplit(n_splits=3,test_size = .3)
cross_val_score(text_clf,X_train,y_train,cv=cv)

array([ 0.79286927,  0.80305603,  0.80814941])

In [20]:
from sklearn.linear_model import SGDClassifier
text_clf = Pipeline([('vect', CountVectorizer()),
                    ('tfidf',TfidfTransformer()),
                    ('clf', SGDClassifier(loss='hinge',
                                          alpha=1e-3,random_state=42))])
cross_val_score(text_clf,X_train,y_train,cv=cv)

array([ 0.79456706,  0.79966044,  0.79966044])

In [21]:
from sklearn.neighbors import KNeighborsClassifier
text_clf = Pipeline([('vect', CountVectorizer()),
                    ('tfidf',TfidfTransformer()),
                    ('clf', KNeighborsClassifier())])
cross_val_score(text_clf,X_train,y_train,cv=cv)

array([ 0.75042445,  0.762309  ,  0.75721562])

In [22]:
from sklearn.linear_model import LogisticRegression
text_clf = Pipeline([('vect', CountVectorizer()),
                    ('tfidf',TfidfTransformer()),
                    ('clf', LogisticRegression())])
cross_val_score(text_clf,X_train,y_train,cv=cv)

array([ 0.76570458,  0.78438031,  0.76061121])

In [23]:
from sklearn.tree import DecisionTreeClassifier
text_clf = Pipeline([('vect', CountVectorizer()),
                    ('tfidf',TfidfTransformer()),
                    ('clf', DecisionTreeClassifier())])
cross_val_score(text_clf,X_train,y_train,cv=cv)

array([ 0.65874363,  0.69779287,  0.67232598])

In [24]:
from sklearn.svm import SVC
text_clf = Pipeline([('vect', CountVectorizer()),
                    ('tfidf',TfidfTransformer()),
                    ('clf', SVC())])
cross_val_score(text_clf,X_train,y_train,cv=cv)

array([ 0.50933786,  0.48896435,  0.50764007])

In [25]:
from collections import Counter
import nltk

def noun_finder (words):
    nouns = []
    words = nltk.Text(words)
    words = nltk.pos_tag(words)
    for w in words:
        if w[1] == 'NN' or w[1]== 'NNS' or w[1]== 'NNP' or w[1]== 'NNPS':
            nouns.append(w[0])
    return nouns

def adj_finder (words):
    adj = []
    words = nltk.Text(words)
    words = nltk.pos_tag(words)
    for w in words:
        if w[1] == 'JJ' or w[1] == 'JJR' or w[1] == 'JJS' or w[1] == 'RB' or w[1]== 'RBR' or w[1]== 'RBS':
            adj.append(w[0])
    return adj

def verb_finder (words):
    verb = []
    words = nltk.Text(words)
    words = nltk.pos_tag(words)
    for w in words:
        if w[1] == 'VB' or w[1]== 'VBD' or w[1]== 'VBG' or w[1]== 'VBN' or w[1]== 'VBP' or w[1]== 'VBZ':
            verb.append(w[0])
    return verb
df['adjecive/adverb'] = df.obj_words.apply(adj_finder)
df['nouns'] = df.obj_words.apply(noun_finder)
df['verbs'] = df.obj_words.apply(verb_finder)

In [26]:
def string_words (words):
    string =' '.join(words)
    if string == '':
        string = 'a'
    return string

df['string_nouns'] = df.nouns.apply(string_words)
df['string_ad'] = df['adjecive/adverb'].apply(string_words)
df['string_verbs'] = df.verbs.apply(string_words)

df.head()

,Review,subject,sentiment,obj_words,all_words,obj_word_count,total_word_count,lem_words,stem_words,word_list,adjecive/adverb,nouns,verbs,string_nouns,string_ad,string_verbs
0,So there is no way for me to plug it in here i...,item,0,"[way, plug, us, unless, go, converter]","[so, there, is, no, way, for, me, to, plug, it...",6,21,"[way, plug, u, unless, go, converter]","[way, plug, us, unless, go, convert]","{'way': True, 'plug': True, 'us': True, 'unles...",[],"[way, plug, converter]",[go],way plug converter,a,go
1,"Good case, Excellent value.\t1",item,1,"[good, case, excellent, value]","[good, case, excellent, value]",4,4,"[good, case, excellent, value]","[good, case, excel, valu]","{'good': True, 'case': True, 'excellent': True...","[good, excellent]","[case, value]",[],case value,good excellent,a
2,Great for the jawbone.\t1,item,1,"[great, jawbone]","[great, for, the, jawbone]",2,4,"[great, jawbone]","[great, jawbon]","{'great': True, 'jawbone': True}",[great],[jawbone],[],jawbone,great,a
3,Tied to charger for conversations lasting more...,item,0,"[tied, charger, conversations, lasting, 45, mi...","[tied, to, charger, for, conversations, lastin...",9,13,"[tied, charger, conversation, lasting, 45, min...","[tie, charger, convers, last, 45, minut, major...","{'tied': True, 'charger': True, 'conversations...",[major],"[charger, conversations, minutes, problems]","[tied, lasting, !!]",charger conversations minutes problems,major,tied lasting !!
4,The mic is great.\t1,item,1,"[mic, great]","[the, mic, is, great]",2,4,"[mic, great]","[mic, great]","{'mic': True, 'great': True}","[mic, great]",[],[],a,mic great,a


In [34]:
X_train, X_test, y_train, y_test = train_test_split(df['Review'],df.sentiment,test_size = .33)

count_vect = CountVectorizer()
x_train_counts = count_vect.fit_transform(X_train)/
tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(x_train_counts)

x_test_counts = count_vect.transform(X_test)
x_test_tfidf = tfidf_transformer.transform(x_test_counts)

text_clf = Pipeline([('vect', CountVectorizer()),
                    ('tfidf',TfidfTransformer()),
                    ('clf', MultinomialNB()),])
clf = MultinomialNB().fit(X_train_tfidf,y_train)
clf.score(x_test_tfidf,y_test)
# cross_val_score(text_clf,X_train,y_train,cv=cv)

  (0, 2970)	1
  (0, 3537)	1
  (0, 163)	1
  (0, 1616)	1
  (0, 2580)	1
  (0, 2364)	2
  (0, 312)	1
  (0, 3433)	1
  (0, 1846)	1
  (0, 1369)	1
  (0, 468)	1
  (0, 3296)	1
  (0, 2003)	1
  (0, 990)	1
  (1, 1287)	1
  (1, 2299)	1
  (1, 3699)	1
  (1, 3418)	1
  (1, 1030)	1
  (1, 2388)	1
  (1, 3394)	1
  (1, 900)	1
  (1, 1655)	1
  (1, 3368)	1
  (2, 1081)	1
  :	:
  (1960, 3385)	1
  (1960, 1480)	1
  (1960, 3731)	1
  (1960, 1643)	1
  (1960, 2299)	3
  (1960, 163)	2
  (1961, 2427)	1
  (1961, 937)	1
  (1961, 2934)	1
  (1961, 1465)	1
  (1961, 1804)	1
  (1961, 3433)	1
  (1962, 238)	1
  (1962, 2859)	1
  (1962, 3521)	1
  (1962, 1158)	1
  (1962, 3652)	1
  (1962, 2856)	1
  (1962, 1895)	1
  (1962, 1135)	1
  (1962, 3382)	1
  (1962, 2209)	1
  (1962, 3370)	1
  (1962, 2317)	1
  (1962, 163)	1


0.81611570247933884